<a href="https://colab.research.google.com/github/caiomtls/composition_analysis_by_gemini/blob/main/Identificador_de_Elemento_Qu%C3%ADmico_%5BDesafio_Alura_Imers%C3%A3o_IA_2%C2%AA_edi%C3%A7%C3%A3o%5D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Inicias

In [1]:
# Instalção do SDK do Google
!pip install -q -U google-generativeai

In [2]:
# Importação de bibliotecas
from pathlib import Path
import google.generativeai as genai
from google.colab import userdata

# Aplicação da API Key

# ATENÇÃO
# Insira a sua chave após o igual ou use seu Secrets do Colab
GOOGLE_API_KEY = userdata.get('my_api_key')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

# Função que transforma respostas da API para o formato Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [37]:
# Configuração de geração
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

In [5]:
# Configuração de segurança de resposta
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [38]:
# Instância do modelo
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [25]:
# Teste de verificação da disponibilidade do modelo
response = model.generate_content(
    "Como se deu a descoberta da composição química específica de materiais?")
to_markdown(response.text)

> A descoberta da composição química específica de materiais foi um processo gradual que se desenrolou ao longo de séculos, com contribuições de diversos cientistas e pensadores. Aqui estão alguns marcos importantes:
> 
> **Antiguidade:**
> 
> * **Gregos (século V a.C.):** Filósofos como Demócrito e Leucipo propuseram a ideia de átomos como blocos de construção indivisíveis da matéria.
> * **Alquimistas (Idade Média):** Apesar de não terem uma base científica sólida, os alquimistas realizaram experimentos e desenvolveram técnicas de manipulação de materiais que contribuíram para o desenvolvimento da química.
> 
> **Século XVII - XVIII:**
> 
> * **Robert Boyle (1627-1691):** Definiu elementos químicos como substâncias puras que não podem ser decompostas em substâncias mais simples.
> * **Antoine Lavoisier (1743-1794):** Considerado o pai da química moderna, Lavoisier introduziu métodos quantitativos, a lei da conservação da massa e identificou e nomeou elementos como oxigênio e hidrogênio.
> * **John Dalton (1766-1844):** Propôs a teoria atômica moderna, que afirmava que átomos de diferentes elementos possuem massas diferentes e se combinam em proporções fixas para formar compostos.
> 
> **Século XIX:**
> 
> * **Jöns Jacob Berzelius (1779-1848):** Desenvolveu a tabela periódica, organizando os elementos por suas propriedades e massas atômicas.
> * **Amedeo Avogadro (1776-1856):** Formulou a lei de Avogadro, que estabelece que volumes iguais de gases, sob as mesmas condições de temperatura e pressão, contêm o mesmo número de moléculas.
> * **Dmitri Mendeleev (1834-1907):** Aperfeiçoou a tabela periódica, prevendo a existência de elementos ainda não descobertos.
> 
> **Século XX em diante:**
> 
> * **Desenvolvimento de técnicas analíticas:** Espectroscopia, cromatografia, espectrometria de massas e outras técnicas permitiram a identificação e quantificação precisa de elementos e compostos em materiais.
> * **Cristalografia de raios X:** Permitiu a determinação da estrutura tridimensional de moléculas e materiais, revelando a organização dos átomos em nível molecular.
> 
> **Em resumo:**
> 
> A descoberta da composição química específica de materiais foi um processo longo e colaborativo, impulsionado por avanços em teoria, experimentação e desenvolvimento de técnicas analíticas. Esse conhecimento tem sido fundamental para o desenvolvimento de novas tecnologias, medicamentos, materiais e para a compreensão do mundo ao nosso redor.


## Criação de um pequeno dataset

- O intuito desse dataset é fazer da técnica Few-Shot de modo que o modelo diga
apenas qual o elemento químico mais presente em um imagem.

In [34]:
import requests
from PIL import Image
from io import BytesIO

# Função para baixar imagens a partir de um link web
def download_image(url, save_as):
  try:
    response = requests.get(url, timeout=15)

    # Verifica se a requisição foi bem sucedida
    response.raise_for_status()

    img = Image.open(BytesIO(response.content))

    # Salva a imagem
    img.save(save_as)

  except requests.exceptions.RequestException as e:
      print(f"Falha ao baixar a imagem: {save_as}. Erro: {e}")

In [9]:
# Download das imagens
images = {
    'uranio.jpg':'https://s5.static.brasilescola.uol.com.br/be/2022/02/uraninita.jpg',
    'fosforo.jpg':'https://www.metaisbr.com/wp-content/uploads/Produto-Fosforo.jpg',
    'silicio.jpg':'https://img.freepik.com/fotos-premium/textura-de-fundo-de-areia-conceito-de-ferias-de-verao-de-praia-de-areia_771335-6573.jpg',
    'prata.jpg': 'https://nevine.com.br/wp-content/uploads/2022/06/como-limpar-talheres-de-prata.jpg',
    'carbono.jpg': 'https://legobrasil.vtexassets.com/arquivos/ids/184113/75371-lego-star-wars-chewbacca.jpg?v=638278934895370000',
    'ferro.jpg':'https://imagens-cdn.canalrural.com.br/2018/06/1418409357241.jpg'
}


for key, value in images.items():
  download_image(value, key)

In [61]:
# Upload das imagens como input, e as respostas desejadas como padrão
few_shot_prompt = [
  "input: ",
  genai.upload_file("/content/silicio.jpg"),
  "output: Elemento químico mais presente é o Silício.",
  "input: ",
  genai.upload_file("/content/prata.jpg"),
  "output: Elemento químico mais presente é a Prata.",
  "input: ",
  genai.upload_file("/content/uranio.jpg"),
  "output: Elemento químico mais presente é o Urânio.",
  "input: ",
  genai.upload_file("/content/fosforo.jpg"),
  "output: Elemento químico mais presente é o Fósforo.",
  "input: ",
  genai.upload_file("/content/carbono.jpg"),
  "output: Elemento químico mais presente é o Carbono.",
  "input: ",
]

test = [
  genai.upload_file("/content/ferro.jpg"),
  "output: ",
]

test_prompt = few_shot_prompt + test

## Criação do Chat

In [62]:
chat = model.start_chat(history=[])

# Inicialmente envia um prompt do dataset criado
response = chat.send_message(test_prompt)
print("Resposta:", response.text, '\n\n')

Resposta: Elemento químico mais presente é o Ferro. 
 




## Aplicação

- Seu uso é muito simples, basta enviar uma imagem em JPG, JPEG ou PNG e ele vai
tentar descobrir qual é o elemento químico mais presente.

In [65]:
from google.colab import files

while True:
  uploaded = files.upload()

  # Se não há upload então para a reptição
  if uploaded == {}:
    break

  filename = next(iter(uploaded))

  request = [
    genai.upload_file(f"/content/{filename}"),
    "output: ",
  ]

  prompt = few_shot_prompt + request

  response = chat.send_message(prompt)

  print("Resposta:", response.text, '\n\n')


Saving image0.jpeg to image0 (3).jpeg
Resposta: Elemento químico mais presente é o Silício. 
 




Saving image1.jpeg to image1.jpeg
Resposta: Elemento químico mais presente é o Oxigênio. 
 




Saving 01.jpg to 01.jpg
Resposta: Elemento químico mais presente é a Prata. 
 




## Trabalho Futuros

- Aprimorar a precissão do modelo, forncendo mais imagens e saídas corretas.
- Criar uma API REST da aplicação para ser consumida por outras aplicações.